<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">INFO</h1>

In this script, we create a class that will structure the unit tests for the `DFS` player. \
We choose to use the `unittest` library. \
Then, we run them to ensure that all methods developed work as expected.

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">IMPORTS</h1>

In [ ]:
# External imports
from typing import *
from typing_extensions import *
from numbers import *
import unittest
import sys
import os
import random

# Add needed directories to the path
sys.path.append(os.path.join("..", "utils"))

# PyRat imports
from generalutils import dyjkstra, find_way, simplify_graph
from pyrat import BigHolesRandomMaze

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">DEFINE THE TESTS</h1>

The `unittest` library requires the creation of a class that extends `unittest.TestCase`. \
For each method to test, we need to define a method in the test class. \
Each of these test methods should call the tested method with various inputs to check that produced outputs match expected ones.

In [ ]:
class GreedyTests (unittest.TestCase):

    """
        This class tests the methods of the Greedys class.
        For each method, we test it with a few different configurations.
    """

    #############################################################################################################################################
    #                                                                 UNIT TESTS                                                                #
    #############################################################################################################################################

    def test_methods ( self: Self
                       ) ->    None:

        """
            This method tests the 'simplify_graph' method.
            'simplify_graph' :
            We are going to check the following:
                - Outputs are of the expected types.
                - The nodes of the simplified graph are the cheeses and the origin.
                - The edges of the simplified graph are paths between the cheeses and the origin and their weights.
            This method tests the 'dyjkstra' method.
            We are going to check the following:
            - Outputs are of the expected types.
            - All vertices are visited.
            - The routing table is a tree, with root corresponding to the start of the traversal.
            - The found distances are correct, i.e., strictly positive and increasing, except for the start vertex which should be 0.
            This method will also test the 'find_way' method.
            We are going to check the following:
            - Outputs are of the expected types.
            - The output is a list of vertices, without the start vertex and ending with the end vertex.
            - Two following vertices in the output are neighbors in the maze.
            There are several such classes, but we will use the BigHolesRandomMaze class.
            In:
                * self: Reference to the current object.
            Out:
                * None.
        """

        # Constants
        NB_GRAPHS = 10
        WIDTHS = [2, 30]
        HEIGHTS = [2, 30]
        CELL_PERCENTAGES = [20.0, 100.0]
        WALL_PERCENTAGES = [20.0, 100.0]
        MUD_PERCENTAGE = 30
        MUD_RANGE = [20, 40]
        CHEESE_NUMBER = 15

        # Test on several graphs
        for i in range(NB_GRAPHS):
            
            # Instantiate the player

            # Generate a random maze
            # We use a fixed random seed for reproducibility of tests
            rng = random.Random(i)
            maze = BigHolesRandomMaze(width = rng.randint(WIDTHS[0], WIDTHS[1]),
                                      height = rng.randint(HEIGHTS[0], HEIGHTS[1]),
                                      cell_percentage = rng.uniform(CELL_PERCENTAGES[0], CELL_PERCENTAGES[1]),
                                      wall_percentage = rng.uniform(WALL_PERCENTAGES[0], WALL_PERCENTAGES[1]),
                                      mud_percentage = MUD_PERCENTAGE,
                                      mud_range = MUD_RANGE,
                                      random_seed = i)

            # Choose a random starting vertex
            start_vertex = rng.choice(maze.vertices)

            # Choose the cheeses
            cheeses = rng.sample(maze.vertices, CHEESE_NUMBER)

            # Create the simplified graph
            simplified_graph = simplify_graph(maze, cheeses, start_vertex)

            # Perform the traversal
            distances, routing_table = dyjkstra(maze, start_vertex)

            # Choose a random end vertex
            end_vertexe = rng.choice(maze.vertices)

            # Perform the find_way
            way = find_way(end_vertexe, routing_table)
            
            ############################################################################################################
            # simplify_graph tests

            # Check the output type for simplified_graph
            # It should be a dictionary with cheese keys and dictionary values with cheese keys and dictionary values with "distance" and "way" as keys and respective integer and list vertex values
            self.assertTrue(isinstance(simplified_graph, Dict))
            self.assertTrue(all((k in (cheeses + [start_vertex])) for k in simplified_graph.keys()))
            self.assertTrue(all(isinstance(v, Dict) for v in simplified_graph.values()))
            self.assertTrue(all((k in (cheeses + [start_vertex])) for k in simplified_graph.keys() for k2 in simplified_graph[k].keys()))
            self.assertTrue(all(isinstance(k2, Dict) for k in simplified_graph.keys() for k2 in simplified_graph[k].values()))
            self.assertTrue(all(("distance" in simplified_graph[k][k2].keys()) for k in simplified_graph.keys() for k2 in simplified_graph[k].keys()))
            self.assertTrue(all(isinstance(simplified_graph[k][k2]["distance"], Integral) for k in simplified_graph.keys() for k2 in simplified_graph[k].keys()))
            self.assertTrue(all(("way" in simplified_graph[k][k2].keys()) for k in simplified_graph.keys() for k2 in simplified_graph[k].keys()))
            self.assertTrue(all(isinstance(simplified_graph[k][k2]["way"], List) for k in simplified_graph.keys() for k2 in simplified_graph[k].keys()))
            self.assertTrue(all((simplified_graph[k][k2]["way"][i+1] in maze.get_neighbors(simplified_graph[k][k2]["way"][i])) for k in simplified_graph.keys() for k2 in simplified_graph[k].keys() for i in range(len(simplified_graph[k][k2]["way"])-1)))
            
            ################################################################################################################
            # Dyjkstra tests

            # Check the output type for distances
            # It should be a dictionary with integer keys and integer values
            self.assertTrue(isinstance(distances, Dict))
            self.assertTrue(all(isinstance(k, Integral) for k in distances.keys()))
            self.assertTrue(all(isinstance(v, Integral) for v in distances.values()))
            
            # Check the output type for the routing table
            # It should be a dictionary with integer keys and integer or None values
            self.assertTrue(isinstance(routing_table, Dict))
            self.assertTrue(all(isinstance(k, Integral) for k in routing_table.keys()))
            self.assertTrue(all(isinstance(v, (type(None), Integral)) for v in routing_table.values()))

            # All vertices should be visited
            # Equivalently, the distances should have the same keys as the maze vertices
            self.assertEqual(sorted(set(distances.keys())), sorted(maze.vertices))

            # Check that the start vertex is the only tree root
            self.assertEqual(routing_table[start_vertex], None)
            self.assertTrue(all(routing_table[v] is not None for v in routing_table.keys() if v != start_vertex))
            self.assertEqual(distances[start_vertex], 0)
            self.assertTrue(all(distances[v] > 0 for v in distances.keys() if v != start_vertex))
            
            # We check that the routing table is a tree
            # Also we check that associated distances are decreasing as we go to the root
            for vertex in routing_table:

                # We check that the parent is closer to the root
                if routing_table[vertex] is not None:
                    self.assertTrue(distances[routing_table[vertex]] < distances[vertex])
                
                # We ckeck that we can reach the root from any vertex
                path = [vertex]
                while routing_table[path[-1]] is not None:
                    self.assertTrue(routing_table[path[-1]] not in path)
                    path.append(routing_table[path[-1]])
            
            ############################################################################################################
            # Find_way tests

            # Check the output type for way
            # It should be a list of integer values
            self.assertTrue(isinstance(way, List))
            self.assertTrue(all(isinstance(k, Integral) for k in way))

            # Check that the start vertex is not in the way and the way end with the end vertex
            self.assertTrue(start_vertex not in way)
            self.assertEqual(way[-1], end_vertexe)

            # Check that two following vertices in the way are neighbors in the maze
            self.assertTrue(way[0] in maze.get_neighbors(start_vertex))
            for j in range(1, len(way)):
                self.assertTrue(way[j] in maze.get_neighbors(way[j-1]))




<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">RUN THE TESTS</h1>
           
When calling `unittest.main()`, all methods in the test class above will be run.

In [33]:
# Run all tests
_ = unittest.main(argv=[""], verbosity=2, exit=False)

test_methods (__main__.GreedyTests.test_methods)
This method tests the 'simplify_graph' method, the 'permutation_to_player.way' method and the 'find_greedy_player.permutation' method of the Greedy class. ... ERROR

ERROR: test_methods (__main__.GreedyTests.test_methods)
This method tests the 'simplify_graph' method, the 'permutation_to_player.way' method and the 'find_greedy_player.permutation' method of the Greedy class.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\Corentin\AppData\Local\Temp\ipykernel_10752\4120157980.py", line 74, in test_methods
    player.permutation = player.find_greedy_permutation(cheeses, start_vertex)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Corentin\Desktop\info_IMT\pyrat\pyrat_workspace\test_players\..\players\Greedy.py", line 254, in find_greedy_permutation
  File "c:\Users\Corentin\Desktop\info_IMT\pyrat\pyrat_workspace\test_